#### tl;dr

A simple test of a library providing json->html data viewing, with browser or inline Razor page, using mocked data.

Solves: effective storage and viewing of divergent (1:n) data sets.

# Json support (Nix-Library-Raw)

This set of code was used to develop a support library to simplify data exploration.

It focuses on **json** data, specifically json Arrays or Objects such as would be expected from a database query using `for json path`. It wraps the conversion of the json string to an html string suitable for presentation.

Because in discovery we are frequently trying to understand relationships and cardinality, it is not usually necessary to retrieve vast quantities of data, rather this allows the display of "several" samples. Nested relationships are enclosed and can be limited to arbitrarily small samples.

A css style sheet provides the underlying styling: this can be altered for different purposes.
Some variables are used, which also allows their modification through Javascript.


## Summary of library functionality

The code is dependent on System.Text.Json (which replaces the older Newtonsoft.Json)

Functionality is wrapped mainly in the `Nest` class. This is a simple class with a `Name`, `Value`, `Nesting` and Children which ate themselves `Nest`s.

### Constructors

For general use, the Nest class is not needed, the static methods handle this for you. However if you need to use your own hierarchical structure, have at it:

`Nest(string Name, string json)` Creates a Nest from a json string representing an array or object.
`Nest(string Name, JsonNode node, int depth = 0)` Is used internally and recursively by the simpler constructor, but can be used to process a JsonObject directly.

The `Nest` class is a simpler and more compact representation than a JsonObject, so can be used to digest and store hierarchial objects for multiple uses.

For one-off rendering, the static methods below are more effective.

### Class Methods

- `string AsHtml(NestOptions options)` returns Html for displaying the object directly.
- `IEnumerable<string>Elements()` returns an enumerable series of strings describing the structure of the data (typically for understanding structure).
- `string AllElements(bool withLineBreaks = false)` returns all the elements with or without line feeds.

### Static methods

- `string Prettify(string json)` formats a json string as indented, fo human consumption.

- `string RenderHtml(string json)` return html suitable for embedding in a page using external css.
- `string RenderPage(string json, NestOptions options)` returns html with css suitable for display as a single page.

- `string GetCss(NestOptions options)` returns the css code for the specified options.

and for convenience:

- `void ToFile(string data, string path)` pushes a string to a local file.
- `string FromFile(string path)` reads a string from a local file.


### Nest Options

To fine-tune the behavior, it is possible to tweak several settings.- 

- `MaximumPixelWidth` sets the width of the enclosing container, to force horizontal scrolling, default 800.
- `TitleColumnWidth` sets the width of the leading column used to label elements, default 120;
- `SpecifyFont` defaults to true, which sets the font to consolas, monospace.
- `SmallerTitles` defaults to true, which reduces the size of titles.
- `ItalicTitles` defaults to true which makes titles italic.
- `ItemLimit` limits the number of items shown in lists, default 10.
- `DarkMode` uses a dark theme, which is the default.



## How about a serious test?

We will use Bogus to generate a slew of information, which we will reduce to json then use to test variations of options.

In [1]:
// load packages
#i "file:///n:/nix-nuget/"
// Library implementing the Nest class.
#r "nuget:nix-library-raw"  
// Used to generate random data for testing
#r "nuget:bogus"            
// Needed by the library and/or jsoin serialization
#r "nuget:System.Text.Json" 
// This allows you to use Razor Pages directly (see later example)
#r "nuget:RazorInteractive" 

Restore sources file:///n:/nix-nuget/ Installed Packages bogus, 35.5.0 nix-library-raw, 1.0.17 RazorInteractive, 1.2.0 System.Text.Json, 8.0.3

Loading extensions from `C:\Users\NickMajor\.nuget\packages\razorinteractive\1.2.0\lib\net8.0\RazorInteractive.dll`

Renders the code block as Razor markup in dotnet-interactive notebooks. 
 This extension adds a new kernel that can render Razor markdown. 
 All C# and F# variables are available in the @Model property.

In [2]:
// make some bogus data
// // This will make a string "json" that has a hierarchial object.
using Bogus;
// Some ranf-ges
int lo = 15;
int hi = 25;

// Let's start with a hierarchial object with some nested arrays.
public class DataSet
{
    public string Client { get; set; }
    public int Size { get; set; }
    public int LocationCount => Locations.Count(); 
    public IEnumerable<Location> Locations { get; set; } = new List<Location>();
}
public class Location
{
    public string LocationName { get; set; }
    public IEnumerable<string> TelephoneNumbers { get; set; } = new List<string>();
    public IEnumerable<string> AddressRows { get; set; } = new List<string>();
        public int EmployeeCount => Employees.Count();
        public IEnumerable<Employee> Employees { get; set; } = new List<Employee>();
}
public class Employee
{
    public int Id { get; set; }
    public string Name { get; set; }
    public string Title { get; set; }
    public string Email { get; set; }
    public string Telephone { get; set; }
}

var count = new Faker().Random.Number(lo, hi);
int identity = 1000;
var dataFaker = new Faker<DataSet>()
    .RuleFor(d => d.Client, f => f.Company.CompanyName())
    .RuleFor(d => d.Size, count)
    .RuleFor(d => d.Locations, f => f.Make(count, () => new Location
    {
        LocationName = f.Address.City(),
        TelephoneNumbers = f.Make(lo, () => f.Phone.PhoneNumber()),
        AddressRows = f.Make(lo, () => f.Address.StreetAddress()),
        Employees = f.Make(f.Random.Number(lo, hi), () => new Employee
        {
            Id = identity++,
            Name = f.Name.FullName(),
            Title = f.Name.JobTitle(),
            Email = f.Internet.Email(),
            Telephone = f.Phone.PhoneNumber()
        })
    }));

    // we make some data...
    var objectData = dataFaker.Generate();
    var arrayData = objectData.Locations;
    // convert the data to a displayable json string
    var objectJson = System.Text.Json.JsonSerializer.Serialize(objectData);
    var arrayJson = System.Text.Json.JsonSerializer.Serialize(arrayData);
    
   // Console.WriteLine("*** JSON OBJECT ***");
   // Nest.Prettify(objectJson).Display();
   // Console.WriteLine("*** JSON ARRAY ***");
   // Nest.Prettify(arrayJson).Display();

## Now the raw library has been added as a Nuget package.

Let's test against that!


In [3]:
using Nix.Library.Raw;
// some options:
var options = new Nest.NestOptions() { DarkMode = true, MaximumListPixelWidth = 1000 };
// generate a page to display the object:
Nest.ToFile(Nest.RenderPageScrolling(objectJson, "Object", 5, "A", options) , "./.html_test/Test-objectX.html");
// generate a page to generate the array:
Nest.ToFile(Nest.RenderPageScrolling(arrayJson, "Array", 5, "B", options ) , "./.html_test/Test-arrayX.html");

In [4]:
#!pwsh
# The two created html files can be launched in the browser dircetly in a powershell script:
./.html_test/Test-objectX.html
./.html_test/Test-arrayX.html

In [5]:
// Or we could use Razor:
#!csharp

// Write a local css file to sytle things:
Nest.ToFile(Nest.GetCss(options), "./local-test.css");
// generate a couple of json strings, one for the object, one for the array:
var A = Nest.RenderHtmlScrolling(objectJson, "Object", 5);
var B = Nest.RenderHtmlScrolling(arrayJson, "Array", 5);

// display both in a razor page directly: (the @Model exposes any variables form NetInteractive)
#!razor
<html>
  <head>
    <link rel="stylesheet" href="./local-test.css">
  </head>
  <body>
    <h3>Object rendered raw in-line</h3>
    @Raw(@Model.A)
    <h3>Array rendered raw in-line</h3>
    @Raw(@Model.B)
    <hr>
  </body>
</html>

Object rendered raw in-line 
 Object 4 elements Client Turcotte, Mayer and Herman 
 Size 20 
 LocationCount 20 
 Locations 
 1/5 2/5 3/5 4/5 5/5 LocationName South Elenor Huelsshire Hudsonchester Uptonchester Nicolettebury TelephoneNumbers 
 631-997-8075 (527) 582-3900 x321 1-383-933-0896 x463 557.385.7083 x72451 (442) 223-4666 x112 (724) 262-6533 x87105 1-282-598-1237 (528) 720-4793 x6805 1-768-205-3415 (600) 263-4689 x828 586.340.4724 x903 1-580-441-7527 x67372 556-211-2732 x48278 (856) 464-6875 1-316-244-4070 
 302.704.8336 x3489 901.471.7444 x7442 605-627-6164 309-673-6156 x978 (664) 485-5866 928-260-7156 x1950 1-480-685-9083 1-736-798-1220 x072 (447) 880-1719 (238) 560-0514 x7532 757-551-0617 x747 1-565-559-6773 361-367-2079 x6903 (945) 222-5201 x1133 1-419-215-2365 x315 
 295.859.8275 966-736-7135 502.389.8512 x2052 251-363-2996 x789 585.462.5388 x2884 662.935.1402 1-594-992-5849 1-444-547-1041 x781 1-613-341-3604 x99996 (760) 405-3979 1-320-678-9014 1-253-555-3792 x66052 366-388-5628 x2980 299.476.1987 x6855 (589) 595-7430 x043 
 1-938-955-9666 x75692 (713) 473-2882 x2306 (729) 203-1485 503.601.8033 238.439.0744 x31397 699-785-3261 364.616.1448 x6622 349-409-6244 x92651 1-746-240-6647 (355) 391-5159 910-510-1266 x4302 1-998-347-3717 420.906.6207 1-682-588-9179 x6772 350.765.4371 
 358-243-2510 x49582 257-998-7647 425-647-1719 x11506 246-806-4924 x982 365-566-5612 (641) 688-7426 x179 1-287-867-3307 1-495-400-7885 x10691 886-297-8293 x195 621-741-9451 x1624 1-503-395-9020 x79801 487.862.1339 1-665-838-1680 1-871-392-0527 328-567-1340 x67105 AddressRows 
 4499 Klein Mountains 359 Pouros Lane 8422 Luettgen Turnpike 09258 Zetta Passage 345 Legros Burgs 95443 Schultz Place 77234 Roob Mountains 2655 Conn Junctions 054 Tillman Row 85912 Legros Ford 245 Watsica Gateway 704 Hettie Locks 8304 Willie Drive 70433 Reinger Pine 7304 Lennie Village 
 226 Jacklyn Hills 176 Heathcote Cliff 3660 Cornell Course 2982 Gerardo Ranch 83308 Crona Forges 265 Steuber View 0510 Adolph Avenue 97939 Thora Lock 51572 Schroeder Shoal 2679 Weissnat Haven 461 Wolf Passage 368 Heaney Ridge 22872 Hilda Knolls 09228 Tyree Rapid 48528 Langosh Glen 
 546 Zboncak Mountains 68474 Norris Mount 76280 Gordon Knoll 505 Alia Junctions 83274 Rice Coves 526 Rolfson Island 426 Smith Corners 12014 Boehm Estates 49151 Jacobi Flats 234 Matt Lake 2383 Breitenberg Village 619 DuBuque Trafficway 17233 Jermain Hollow 63998 Lorenz Park 123 Chad Wells 
 14746 Lakin Mews 06087 Torp Divide 5668 Homenick Inlet 42988 Shirley Skyway 75625 Abbigail Prairie 073 Zboncak Springs 6171 Nicolas Forge 50030 Jones Roads 3763 Ondricka Avenue 4485 Zechariah Camp 195 Roberts Ramp 641 Heaven Springs 321 Jeff Center 96121 Kuvalis Mountain 821 Carlo Path 
 83074 Heaney Mountains 284 Vicky Skyway 438 Jessika Ford 7594 Frami Hill 5010 Tristin Falls 69953 Kasandra Haven 09652 Eugenia Villages 30441 Elliott Prairie 2012 Abe Circle 51787 Hannah Mall 0288 Ezekiel Mews 9690 Greenfelder Overpass 059 Abelardo Extension 946 Fay Burg 8286 Casper Bridge EmployeeCount 20 23 20 16 16 Employees Employees 
 1/5 2/5 3/5 4/5 5/5 Id 1000 1001 1002 1003 1004 Name Thalia Ferry Opal Goyette Emanuel Daniel Brice Sporer Haskell Ortiz Title Global Program Technician Forward Quality Coordinator International Tactics Producer District Solutions Officer Internal Infrastructure Planner Email Isaias_Paucek@hotmail.com Ayden.Koss@hotmail.com Kianna.Strosin@gmail.com Conner.McDermott@yahoo.com Jaden47@yahoo.com Telephone (276) 418-9965 (693) 820-1198 (500) 685-5306 x12238 298.625.4110 1-922-419-3799 x700 Employees 
 1/5 2/5 3/5 4/5 5/5 Id 1020 1021 1022 1023 1024 Name Nicolas Gerlach Nayeli Mayer Claudine Durgan Alysson Gaylord Lacy Wintheiser Title District Accountability Associate Direct Data Director Product Configuration Orchestrator Customer Communications Coordinator Product Branding Strategist Email Jayne_Bartell@gmail.com Tillman.Reichel41@yahoo.com Vickie.Ritchie0@hotmail.com Laurine_Altenwerth36@yahoo.